# 🚑 AmbuRoute - Model Training
## Phase 3: YOLOv5 Ambulance Detection Model Training

This notebook handles the training of the YOLOv5 model for real-time ambulance detection in traffic scenarios.


In [1]:
# Import necessary libraries
import os
import sys
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import yaml
import json
from tqdm import tqdm
import time
import warnings
warnings.filterwarnings('ignore')

# Import YOLOv5
from ultralytics import YOLO
import cv2

# Set up plotting
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("🤖 Model Training Environment Ready!")
print(f"🔧 PyTorch version: {torch.__version__}")
print(f"🎮 CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"🎮 GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")


🤖 Model Training Environment Ready!
🔧 PyTorch version: 2.8.0+cpu
🎮 CUDA available: False


## 🎯 Ambulance Detection Trainer Class


In [3]:
class AmbulanceDetectionTrainer:
    """Comprehensive trainer class for ambulance detection using YOLOv5"""
    
    def __init__(self, project_root="."):
        self.project_root = Path(project_root)
        self.dataset_path = self.project_root / "dataset"
        self.models_path = self.project_root / "models"
        self.results_path = self.project_root / "results"
        
        # Training configuration
        self.config = {
            'model_size': 'yolov5s',  # yolov5n, yolov5s, yolov5m, yolov5l, yolov5x
            'img_size': 640,
            'batch_size': 16,
            'epochs': 100,
            'patience': 20,
            'device': '0' if torch.cuda.is_available() else 'cpu',
            'workers': 4,
            'optimizer': 'AdamW',
            'lr0': 0.01,
            'lrf': 0.1,
            'momentum': 0.937,
            'weight_decay': 0.0005,
            'warmup_epochs': 3,
            'warmup_momentum': 0.8,
            'warmup_bias_lr': 0.1,
            'box': 0.05,
            'cls': 0.5,
            'dfl': 1.5,
            'pose': 12.0,
            'kobj': 1.0,
            'label_smoothing': 0.0,
            'nbs': 64,
            'overlap_mask': True,
            'mask_ratio': 4,
            'dropout': 0.0,
            'val': True,
            'split': 'val',
            'save_json': False,
            'save_hybrid': False,
            'conf': 0.001,
            'iou': 0.6,
            'max_det': 300,
            'half': False,
            'dnn': False,
            'plots': True
        }
        
        # Initialize model
        self.model = None
        self.training_results = None
        self.validation_results = None
        
    def create_dataset_yaml(self):
        """Create YOLO dataset configuration file"""
        dataset_config = {
            'path': str(self.dataset_path.absolute()),
            'train': 'images/train',
            'val': 'images/val',
            'test': 'images/test',
            'nc': 1,  # number of classes
            'names': ['ambulance']  # class names
        }
        
        yaml_path = self.dataset_path / 'ambulance_dataset.yaml'
        with open(yaml_path, 'w') as f:
            yaml.dump(dataset_config, f, default_flow_style=False)
        
        print(f"✅ Dataset YAML created: {yaml_path}")
        return yaml_path
    
    def load_pretrained_model(self):
        """Load pre-trained YOLOv5 model"""
        print(f"🔄 Loading pre-trained {self.config['model_size']} model...")
        
        try:
            self.model = YOLO(f"{self.config['model_size']}.pt")
            print(f"✅ Model loaded successfully!")
            print(f"📊 Model parameters: {sum(p.numel() for p in self.model.model.parameters()):,}")
            return True
        except Exception as e:
            print(f"❌ Error loading model: {e}")
            return False
    
    def start_training(self, custom_config=None):
        """Start the training process"""
        if custom_config:
            self.config.update(custom_config)
        
        print("🚀 Starting training process...")
        print(f"📊 Training configuration:")
        for key, value in self.config.items():
            print(f"   {key}: {value}")
        
        try:
            # Create dataset YAML
            dataset_yaml = self.create_dataset_yaml()
            
            # Start training
            start_time = time.time()
            
            self.training_results = self.model.train(
                data=str(dataset_yaml),
                epochs=self.config['epochs'],
                batch=self.config['batch_size'],
                imgsz=self.config['img_size'],
                device=self.config['device'],
                workers=self.config['workers'],
                patience=self.config['patience'],
                save=True,
                exist_ok=True,
                project=str(self.results_path),
                name='ambulance_detection',
                optimizer=self.config['optimizer'],
                lr0=self.config['lr0'],
                lrf=self.config['lrf'],
                momentum=self.config['momentum'],
                weight_decay=self.config['weight_decay'],
                warmup_epochs=self.config['warmup_epochs'],
                warmup_momentum=self.config['warmup_momentum'],
                warmup_bias_lr=self.config['warmup_bias_lr'],
                box=self.config['box'],
                cls=self.config['cls'],
                dfl=self.config['dfl'],
                pose=self.config['pose'],
                kobj=self.config['kobj'],
                label_smoothing=self.config['label_smoothing'],
                nbs=self.config['nbs'],
                overlap_mask=self.config['overlap_mask'],
                mask_ratio=self.config['mask_ratio'],
                dropout=self.config['dropout'],
                val=self.config['val'],
                split=self.config['split'],
                save_json=self.config['save_json'],
                save_hybrid=self.config['save_hybrid'],
                conf=self.config['conf'],
                iou=self.config['iou'],
                max_det=self.config['max_det'],
                half=self.config['half'],
                dnn=self.config['dnn'],
                plots=self.config['plots']
            )
            
            training_time = time.time() - start_time
            print(f"✅ Training completed in {training_time/3600:.2f} hours!")
            
            return True
            
        except Exception as e:
            print(f"❌ Training failed: {e}")
            return False
    
    def evaluate_model(self, model_path=None):
        """Evaluate the trained model"""
        if model_path is None:
            model_path = self.results_path / 'ambulance_detection' / 'weights' / 'best.pt'
        
        if not Path(model_path).exists():
            print(f"❌ Model not found at {model_path}")
            return None
        
        print("🔍 Evaluating model...")
        
        try:
            # Load the best model
            eval_model = YOLO(str(model_path))
            
            # Run validation
            self.validation_results = eval_model.val(
                data=str(self.dataset_path / 'ambulance_dataset.yaml'),
                split='val',
                batch=self.config['batch_size'],
                imgsz=self.config['img_size'],
                conf=self.config['conf'],
                iou=self.config['iou'],
                max_det=self.config['max_det'],
                half=self.config['half'],
                device=self.config['device'],
                workers=self.config['workers'],
                plots=True,
                save_json=True,
                save_hybrid=False
            )
            
            print("✅ Model evaluation completed!")
            return self.validation_results
            
        except Exception as e:
            print(f"❌ Evaluation failed: {e}")
            return None
    
    def plot_training_results(self):
        """Plot training results and metrics"""
        if self.training_results is None:
            print("❌ No training results available")
            return
        
        print("📊 Plotting training results...")
        
        # Create comprehensive plots
        fig, axes = plt.subplots(2, 3, figsize=(18, 12))
        fig.suptitle('AmbuRoute Training Results', fontsize=16, fontweight='bold')
        
        # Training loss
        if hasattr(self.training_results, 'results_dict'):
            results = self.training_results.results_dict
        else:
            results = self.training_results
        
        # Plot 1: Training Loss
        if 'train/box_loss' in results:
            axes[0,0].plot(results['train/box_loss'], label='Box Loss', color='blue')
        if 'train/obj_loss' in results:
            axes[0,0].plot(results['train/obj_loss'], label='Object Loss', color='red')
        if 'train/cls_loss' in results:
            axes[0,0].plot(results['train/cls_loss'], label='Class Loss', color='green')
        axes[0,0].set_title('Training Loss')
        axes[0,0].set_xlabel('Epoch')
        axes[0,0].set_ylabel('Loss')
        axes[0,0].legend()
        axes[0,0].grid(True)
        
        # Plot 2: Validation Metrics
        if 'val/box_loss' in results:
            axes[0,1].plot(results['val/box_loss'], label='Val Box Loss', color='blue')
        if 'val/obj_loss' in results:
            axes[0,1].plot(results['val/obj_loss'], label='Val Object Loss', color='red')
        if 'val/cls_loss' in results:
            axes[0,1].plot(results['val/cls_loss'], label='Val Class Loss', color='green')
        axes[0,1].set_title('Validation Loss')
        axes[0,1].set_xlabel('Epoch')
        axes[0,1].set_ylabel('Loss')
        axes[0,1].legend()
        axes[0,1].grid(True)
        
        # Plot 3: Precision and Recall
        if 'metrics/precision(B)' in results:
            axes[0,2].plot(results['metrics/precision(B)'], label='Precision', color='purple')
        if 'metrics/recall(B)' in results:
            axes[0,2].plot(results['metrics/recall(B)'], label='Recall', color='orange')
        axes[0,2].set_title('Precision & Recall')
        axes[0,2].set_xlabel('Epoch')
        axes[0,2].set_ylabel('Score')
        axes[0,2].legend()
        axes[0,2].grid(True)
        
        # Plot 4: mAP
        if 'metrics/mAP50(B)' in results:
            axes[1,0].plot(results['metrics/mAP50(B)'], label='mAP@0.5', color='green')
        if 'metrics/mAP50-95(B)' in results:
            axes[1,0].plot(results['metrics/mAP50-95(B)'], label='mAP@0.5:0.95', color='red')
        axes[1,0].set_title('Mean Average Precision')
        axes[1,0].set_xlabel('Epoch')
        axes[1,0].set_ylabel('mAP')
        axes[1,0].legend()
        axes[1,0].grid(True)
        
        # Plot 5: Learning Rate
        if 'lr/pg0' in results:
            axes[1,1].plot(results['lr/pg0'], label='Learning Rate', color='blue')
        axes[1,1].set_title('Learning Rate Schedule')
        axes[1,1].set_xlabel('Epoch')
        axes[1,1].set_ylabel('Learning Rate')
        axes[1,1].legend()
        axes[1,1].grid(True)
        
        # Plot 6: Model Performance Summary
        if self.validation_results is not None:
            metrics = ['Precision', 'Recall', 'mAP@0.5', 'mAP@0.5:0.95']
            values = [
                getattr(self.validation_results, 'box', {}).get('mp', 0),
                getattr(self.validation_results, 'box', {}).get('mr', 0),
                getattr(self.validation_results, 'box', {}).get('map50', 0),
                getattr(self.validation_results, 'box', {}).get('map', 0)
            ]
            
            bars = axes[1,2].bar(metrics, values, color=['blue', 'red', 'green', 'orange'])
            axes[1,2].set_title('Final Model Performance')
            axes[1,2].set_ylabel('Score')
            axes[1,2].tick_params(axis='x', rotation=45)
            
            # Add value labels on bars
            for bar, value in zip(bars, values):
                height = bar.get_height()
                axes[1,2].text(bar.get_x() + bar.get_width()/2., height + 0.01,
                              f'{value:.3f}', ha='center', va='bottom')
        
        plt.tight_layout()
        plt.show()
        
        print("✅ Training results plotted successfully!")
    
    def save_model(self, model_path=None):
        """Save the trained model"""
        if model_path is None:
            model_path = self.models_path / 'trained' / 'ambulance_detector.pt'
        
        model_path = Path(model_path)
        model_path.parent.mkdir(parents=True, exist_ok=True)
        
        try:
            # Copy the best model from training results
            best_model_path = self.results_path / 'ambulance_detection' / 'weights' / 'best.pt'
            if best_model_path.exists():
                import shutil
                shutil.copy2(best_model_path, model_path)
                print(f"✅ Model saved to: {model_path}")
                return str(model_path)
            else:
                print(f"❌ Best model not found at {best_model_path}")
                return None
        except Exception as e:
            print(f"❌ Error saving model: {e}")
            return None

# Initialize trainer
trainer = AmbulanceDetectionTrainer()
print("✅ Ambulance Detection Trainer initialized!")


✅ Ambulance Detection Trainer initialized!
